In [3]:
import requests , nltk , time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from pandas import DataFrame
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
keyword = 'strawberry'
url = 'https://www.google.com/search?q='+keyword+'&hl=en&source=lnms&tbm=nws&sa=X&ved=0ahUKEwi39q3H_ZjdAhXnMewKHbszDXIQ_AUICygC&biw=1536&bih=728&dpr=1.25'
source_list = []
date_list = []
title_list = []
description_list = []
driver = webdriver.Chrome()

while True:
    try:
        #Parse the webpage
        response = requests.get(url)
        soup = BeautifulSoup(response.text , 'html5lib')
        
        #Grab the Headline
        for title in soup.find_all('h3'):
            title_list.append(title.text)
        
        #Grab the Soruce and Date
        for sd in soup.find_all('div',{'class':'slp'}):
            info = sd.text.split(' - ')
            source = info[0]
            source_list.append(source)
            date = info[len(info)-1]
            date_list.append(date)
        
        #Grab the Description
        for desc in soup.find_all('div',{'class':'st'}):
            description_list.append(desc.text)
        
        #Get the next page url
        driver.get(url)
        driver.find_element_by_link_text('Next').click()
        url = driver.current_url
        
        #Some delay in order to not get blocked [Remove it if you don't have too much pages to review]
        time.sleep(30)
        
    except NoSuchElementException:
        print("No more pages left!")
        break

No more pages left!


In [5]:
def getSentiment(text):
    sentiment = ""
    ps = analyzer.polarity_scores(text)
    
    #Positive sentiments
    if ps['neg'] <= 0.5 and ps['pos'] > 0:
        if ps['pos'] - ps['neg'] >= 0:
            sentiment = '1'
        elif ps['neu'] >= 0.5:
            sentiment = '1'
        else:
            sentiment = '0'
    
    #Negative sentiments
    elif ps['pos'] <= 0.5 and ps['neg'] > 0:
        if ps['neg'] - ps['pos'] >= 0:
            sentiment = '0'
        elif ps['neu'] <= 0.5:
            sentiment = '0'
        else:
            sentiment = '1'
    
    else:
        sentiment = getSentiment('Good '+ text)
    
    return sentiment

In [289]:
x = 'Strawberry cure for painful illnesses'
print(analyzer.polarity_scores(x))
print(getSentiment(x))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
1


In [263]:
i = 0
labels_list = []

analyzer = SentimentIntensityAnalyzer()

x = description_list[1]

for sentence in title_list:
    sentiment = getSentiment(sentence)
    if(sentiment == '1' or sentiment == '0'):
        labels_list.append(sentiment)
    elif(sentiment == '?'):
        labels_list.append(getSentiment(description_list[i]))
    else:
        labels_list.append('?')
        
    i += 1
        
print("Positive News Percentage : " , labels_list.count('1')/len(description_list)*100 , "%")
print("Negative News Percentage : " , labels_list.count('0')/len(description_list)*100 , "%")
print("Unknowns : " , labels_list.count('?') ,'sentences , ' , labels_list.count('?')/len(description_list)*100 , "%")

Positive News Percentage :  87.5 %
Negative News Percentage :  12.5 %
Unknowns :  0 sentences ,  0.0 %


In [260]:
df = DataFrame({'Headline': title_list , 'Source': source_list , 'Date/Time': date_list , 'Description': description_list , 'Labels': labels_list})
df.to_excel('f.xlsx' , sheet_name = 'sheet1' , index = False)

In [9]:
import nltk
import sentiment_mod as s

ModuleNotFoundError: No module named 'sentiment_mod'

In [9]:
analyzer = SentimentIntensityAnalyzer()
x = 'crops'
print(analyzer.polarity_scores(x))
print(getSentiment(x))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
1
